In [1]:
%matplotlib notebook
import numpy as np
import pandas
import matplotlib.pyplot as plt
import os

Let's import the SED fitting code which use `prospector` package.

In [2]:
from pyprosp import prospector

# Data

For this tutorial, we will fit the SED based on some photometric measurements.<br>
What's important with input data is the names you give:

- measurements must be on the format "instrument.band" (e.g. sdss.u, ps1.g, ...)
- uncertainties must be on the format "instrument.band.err" (e.g. sdss.u.err, ps1.g.err, ...)

The measurement uncertainties are necessary for the code to automatically recognize the filters used.<br>
Here we set a `pandas.Series` with data, as it could come from a `pandas.DataFrame` in a loop, but it also works with a dictionary.

These data are coming from "Johnson et al. 2013", the studied galaxy is "NGC4163", measured in maggies.

In [3]:
data = {"SN_name":"NGC4163",
        "zcmb":0.000551,
        "GALEX.FUV":5.91561634e-08, 
        "GALEX.FUV.err":5.91561634e-09, 
        "GALEX.NUV":8.87156012e-08, 
        "GALEX.NUV.err":8.87156012e-09, 
        "sdss.u":2.22843515e-07,
        "sdss.u.err":2.22843515e-08,
        "sdss.g":4.87528490e-07,
        "sdss.g.err":4.87528490e-08,
        "sdss.r":6.98232404e-07,
        "sdss.r.err":6.98232404e-08,
        "sdss.i":8.79022517e-07,
        "sdss.i.err":8.79022517e-08,
        "sdss.z":9.46237161e-07,
        "sdss.z.err":9.46237161e-08, 
        "Spitzer.1":4.32513831e-07, 
        "Spitzer.1.err":4.32513831e-08, 
        "Spitzer.2":2.80543364e-07, 
        "Spitzer.2.err":2.80543364e-08, 
        "Spitzer.3":1.95884467e-07, 
        "Spitzer.3.err":1.95884467e-08, 
        "Spitzer.4":1.29419584e-07, 
        "Spitzer.4.err":1.29419584e-08, 
        }

data = pandas.Series(data)
data

SN_name              NGC4163
zcmb                0.000551
GALEX.FUV        5.91562e-08
GALEX.FUV.err    5.91562e-09
GALEX.NUV        8.87156e-08
GALEX.NUV.err    8.87156e-09
sdss.u           2.22844e-07
sdss.u.err       2.22844e-08
sdss.g           4.87528e-07
sdss.g.err       4.87528e-08
sdss.r           6.98232e-07
sdss.r.err       6.98232e-08
sdss.i           8.79023e-07
sdss.i.err       8.79023e-08
sdss.z           9.46237e-07
sdss.z.err       9.46237e-08
Spitzer.1        4.32514e-07
Spitzer.1.err    4.32514e-08
Spitzer.2        2.80543e-07
Spitzer.2.err    2.80543e-08
Spitzer.3        1.95884e-07
Spitzer.3.err    1.95884e-08
Spitzer.4         1.2942e-07
Spitzer.4.err     1.2942e-08
dtype: object

# Prospector SED fitting

### Initialization

Now that we have data to work with, let's build a `prospector` object.

In [4]:
prosp = prospector.Prospector()

Prospector can work either with measured photometry or spectrum (or with both).<br>
Here we give the data we constructed above (`phot=data`). Note that no matter there are some additional informations in these data, the code ignore them, only photometry is managed (with the constraints on name format highlighted above).<br>
We let `spec` to None (`spec=None`, default value).
We inform the code of the measurement unit (`unit="mgy"`). Note that there are other available units treated by the code:
- "Hz": erg/s/cm2/Hz
- "AA": erg/s/cm2/AA
- "mgy": maggies
- "Jy": Jansky
- "mag": magnitudes

This unit defines the your measurements in general, whether you give photometry or spectrometry. However if you give both, you cannot define a different unit for each one. They must be in the same unit.

The builder also accept a redshift value to fix it in the SED fitting. If `z` is set to None, the redshift becomes a free parameter for the fit.<br>
Finally, you can optionally give an object name.

In [5]:
prosp.set_data(phot=data, spec=None, unit="mgy", z=data.zcmb, name=data.SN_name)

# Can be executed directly in the builder:
# prosp = prospector.Prospector(phot=data, spec=None, unit="mgy", z=data.zcmb, name=data.SN_name)

`prospector` needs now three specific objects to be built: 
- the `obs` containing every measurements
- the `model` synthesizing the parameters defining the SED model. The free paramters of the model will be fitted during SED fitting.
- the `sps` refering to the Stellar Population Synthesis, calling FSPS procedures.

It doesn't matter in which order you build `obs` and `model`, but `sps` has to be built after the `model` (because it depends on the paramters implemented in the `model`).

### Build `obs`

Firstly, let's build the `obs` object, which basically is a dictionary formatted to be used in `prospector`.

In this example, we are working with photometry. We may want to avoid some filter measurements during the SED fitting. This can be done by giving a list of filters to be used for the fit to `phot_mask` argument (there is an example in next cell in comment). Filters must be on the format "instrument.band" (e.g. sdss.u, ps1.g, ...).

A few options allow to print some user messages:
- `verbose`: print the built `obs` dictionary (default is False)
- `warning`: print warning messages if there's any, noticing some anomalies but without breaking the code

In addition of this example, if you are working on spectroscopy, you can also apply a mask on the measured spectrum by giving limits to `spec_mask` argument in Angstroms (a few examples below).

Finally, if you already have an `obs` prospector compatible dictionary, you can give it in `obs` argument. In this case, `set_data` allow you to choose if you want to save the data from this dictionary in your instance's attributes.

In [6]:
# Example of photometric mask:
# phot_mask = ["sdss.u", "sdss.g", "sdss.r", "sdss.i", "sdss.z"]

# Examples of spectroscopic masks:
# spec_mask = None ; (None, None) ; (1000, None) ; (1e3, 1e5) ; ...

prosp.build_obs(phot_mask=None, verbose=True, warnings=True)#, spec_mask=(None, None), obs=None, set_data=False)


#===============#
#   Built obs   #
#===============#

{'filternames': ['galex_FUV',
                 'galex_NUV',
                 'sdss_u0',
                 'sdss_g0',
                 'sdss_r0',
                 'sdss_i0',
                 'sdss_z0',
                 'spitzer_irac_ch1',
                 'spitzer_irac_ch2',
                 'spitzer_irac_ch3',
                 'spitzer_irac_ch4'],
 'filters': [<class 'sedpy.observate.Filter'>(galex_FUV),
             <class 'sedpy.observate.Filter'>(galex_NUV),
             <class 'sedpy.observate.Filter'>(sdss_u0),
             <class 'sedpy.observate.Filter'>(sdss_g0),
             <class 'sedpy.observate.Filter'>(sdss_r0),
             <class 'sedpy.observate.Filter'>(sdss_i0),
             <class 'sedpy.observate.Filter'>(sdss_z0),
             <class 'sedpy.observate.Filter'>(spitzer_irac_ch1),
             <class 'sedpy.observate.Filter'>(spitzer_irac_ch2),
             <class 'sedpy.observate.Filter'>(spitzer_irac_ch3),
    

### Build `model`

This is the trickiest part of the initialization as there are a lot of possibilities highly impacting the SED fitting.

The `model` stores every paramters defining the model which will fit the data.<br>
Before any treatment, the input parameters are dictionaries including some specific keys:
- `name`: the parameter's name
- `N`: parameter's length (vector parameters are possible)
- `init`: the initial value
- `isfree`: a boolean defining the parameter to free (for SED fitting) or fixed to the `init` value

If the parameter is free (`isfree=True`), you finally need to define:
- `prior`: the prior probability to apply to the parameter during SED fitting

There are a few other optionnal parameters, like:
- `units`: unit of the parameter
- `init_disp`: the initial dispersion to use when generating clouds of emcee "walkers"
- `disp_floor`: the minimum dispersion to use when generating clouds of emcee "walkers".


Here is an example (creating a model with only a "stellar mass" like parameter):

In [7]:
model_tuto = {"mass_tuto":{"N": 1,
                           "isfree": True,
                           "init": 1e8,
                           "prior": dict(name="LogUniform", mini=1e6, maxi=1e12),
                           "init_disp": 1e6, 
                           "disp_floor": 1e6, 
                           "units": "solar masses formed"}, 
              # ... : {...}, 
              }

# Could also be: 
# model_tuto = [{"name":"mass_tuto", 
#                "N": 1,
#                "isfree": True,
#                "init": 1e8,
#                "prior": dict(name="LogUniform", mini=1e6, maxi=1e12),
#                "init_disp": 1e6, 
#                "disp_floor": 1e6, 
#                "units": "solar masses formed"}, 
#               {...}, 
#              ]

About priors, the code will automatically build an appropriate prior instance based on a dictionary containing every needed arguments.

In the parameter `mass_tuto`, `prior = dict(name="LogUniform", mini=1e6, maxi=1e12)`. You can get more informations from this stuff with:

In [8]:
# prosp.describe_priors("LogUniform")
prosp.describe_priors(dict(name="LogUniform", mini=1e6, maxi=1e12))

# If you want to print every avalaible prior informations, run: 
# prosp.describe_priors("*")


#========================#
#   Prior descriptions   #
#========================#

Available priors: TopHat, Normal, ClippedNormal, LogNormal, LogUniform, Beta, StudentT, SkewNormal.

(Required arguments must be included in addition with the 'name' argument in a dictionary to correctly build the prior, 
 for example, try to describe 'priors={'name':'Normal', 'mean':0., 'sigma':1.}').


+----------------+
|   LogUniform   |
+----------------+

Like log-normal, but the distribution of natural log of the variable is
    distributed uniformly instead of normally.

    - mini: (your input is: 1000000.0)
        Minimum of the distribution

    - maxi: (your input is: 1000000000000.0)
        Maximum of the distribution
    


However, instead of building a model from scratch by hand, `prospector` propose a set of pre-packaged parameter sets.

You can have a look at them with:

In [9]:
# You can have a full description of every templates with:
# prosp.describe_templates("*")

# We will limit the print length with the description of the two templates we'll use:
prosp.describe_templates(["parametric_sfh", "dust_emission"])


#=========================#
#   Availbale templates   #
#=========================#

'type_defaults':
  Explicitly sets dust amd IMF types.
'ssp':
  Basic set of (free) parameters for a delta function SFH
'parametric_sfh':
  Basic set of (free) parameters for a delay-tau SFH.
'dust_emission':
  The set of (fixed) dust emission parameters.
'nebular':
  The set of nebular emission parameters, with gas_logz tied to stellar logzsol.
'nebular_marginalization':
  Marginalize over emission amplitudes line contained inthe observed spectrum
'fit_eline_redshift':
  Fit for the redshift of the emission lines separatelyfrom the stellar redshift
'outlier_model':
  The set of outlier (mixture) models for spectroscopy and photometry
'agn':
  The set of (fixed) AGN dusty torus emission parameters.
'igm':
  The set of (fixed) IGM absorption parameters.
'spectral_smoothing':
  Set of parameters for spectal smoothing.
'optimize_speccal':
  Set of parameters (most of which are fixed) for optimizing a pol

With all these elements, let's now build the `model`.<br>
Once again, the instance method `build_model` includes a few arguments.
- `model`: if you created your own model you can add it here (with our example, we add `model_tuto` and see below how to remove it)
- `templates`: here you can give one or a list of template names to build the model. be careful of the order of the names in the list, because the code is using `dict.update` function in the order of the list.
- `describe`: print the description of the priors from the given model (if any) and the given templates (if any).
- `verbose`: print the built `model`

The code starts to build the model with the given one, and then update it with the given templates (if any). If no initial model is given (thus only templates), it starts from the first given template, and then update it with the following ones.

In [10]:
prosp.build_model(model=model_tuto, templates=["parametric_sfh", "dust_emission"], describe=False, verbose=True)


#=================#
#   Built model   #
#=================#

{'add_dust_emission': {'N': 1, 'init': True, 'isfree': False},
 'dust2': {'N': 1,
           'init': 0.6,
           'isfree': True,
           'prior': <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=2.0),
           'units': 'optical depth at 5500AA'},
 'dust_type': {'N': 1, 'init': 0, 'isfree': False},
 'duste_gamma': {'N': 1,
                 'init': 0.001,
                 'isfree': False,
                 'prior': <class 'prospect.models.priors.LogUniform'>(mini=0.001,maxi=0.15),
                 'units': 'Mass fraction of dust in high radiation intensity.'},
 'duste_qpah': {'N': 1,
                'init': 4.0,
                'isfree': False,
                'prior': <class 'prospect.models.priors.TopHat'>(mini=0.5,maxi=7.0),
                'units': 'Percent mass fraction of PAHs in dust.'},
 'duste_umin': {'N': 1,
                'init': 1.0,
                'isfree': False,
                'prior': <class 'pr

Once the model is built, we can still modify it, using `modify_model`.

In this example, we built the model from a handly made one with a tutorial parameter `mass_tuto`. In practical, templates already include a well defined stellar mass parameter (in some of the templates, it even depends on other parameters...). Thus, here, we want to remove `mass_tuto` from the model. We can do that giving a list of parameters to remove from the model to the `removings` argument.

*Note: One could say that we could run again the previous cell without giving `model_tuto`to `model` argument. Indeed, but we want here to show the possiblities of the code...*

Additionaly, depending on the object we are studying, we may want to change the prior probability for some of the parameters, or the initial value, etc. This can be done giving a dictionary very similar to the model creation (see `model_tuto` creation), but only containing parameters with new values. Then, the code applies the changes giving this dictionary to the `changes` argument.

*Note: Two important things coming with the `changes` argument: if there is one (or more) parameter in the given dictionary not existing yet in the model, the code add it to the model ; if there is one (or more) key in one parameter of the given dictionary not existing yet in the same parameter from the model, the code add it to the model.*

The code apply changes before the removings...

Here again, a few more arguments:
- `describe`: print the description of the priors from the given changes (if any).
- `verbose`: print the previous `model` and the final modified one.
- `warnings`: print warning messages if there's any, noticing some anomalies but without breaking the code

In [11]:
changes = {"dust2":{"init":0.05}, 
           "tage":{"init":13., "disp_floor":1.},
           "mass":{"init":1e8, "prior":{"name":"LogUniform", "mini":1e6, "maxi":1e10}, 
                   "init_disp": 1e6, "disp_floor":1e6},
           "tau":{"prior":{"name":"LogUniform", "mini":1e-1, "maxi":1e2}, "disp_floor":1.}, 
           }

removings = ["mass_tuto", "not_existing_parameter"]
# Note that we add in the list of removings a parameter which doesn't exist in the model, 
# but the code just inform you that it already doesn't exist by a warning 
# (which can even be not printed if 'warnings = False').

prosp.modify_model(changes=changes, removings=removings, verbose=True, warnings=True, describe=False)


#====================#
#   Previous model   #
#====================#

{'add_dust_emission': {'N': 1,
                       'init': True,
                       'isfree': False,
                       'name': 'add_dust_emission'},
 'dust2': {'N': 1,
           'init': 0.6,
           'isfree': True,
           'name': 'dust2',
           'prior': <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=2.0),
           'units': 'optical depth at 5500AA'},
 'dust_type': {'N': 1, 'init': 0, 'isfree': False, 'name': 'dust_type'},
 'duste_gamma': {'N': 1,
                 'init': 0.001,
                 'isfree': False,
                 'name': 'duste_gamma',
                 'prior': <class 'prospect.models.priors.LogUniform'>(mini=0.001,maxi=0.15),
                 'units': 'Mass fraction of dust in high radiation intensity.'},
 'duste_qpah': {'N': 1,
                'init': 4.0,
                'isfree': False,
                'name': 'duste_qpah',
                'prior': <class 'prospec

/Users/mart1/anaconda3/lib/python3.7/site-packages/pykcorr-0.2.1-py3.7.egg/pyprosp/prospector.py:412: UserWarning: 'disp_floor' is not included in 'tage' model parameters. Adding it...
/Users/mart1/anaconda3/lib/python3.7/site-packages/pykcorr-0.2.1-py3.7.egg/pyprosp/prospector.py:412: UserWarning: 'init_disp' is not included in 'mass' model parameters. Adding it...
/Users/mart1/anaconda3/lib/python3.7/site-packages/pykcorr-0.2.1-py3.7.egg/pyprosp/prospector.py:412: UserWarning: 'disp_floor' is not included in 'mass' model parameters. Adding it...
/Users/mart1/anaconda3/lib/python3.7/site-packages/pykcorr-0.2.1-py3.7.egg/pyprosp/prospector.py:412: UserWarning: 'disp_floor' is not included in 'tau' model parameters. Adding it...
/Users/mart1/anaconda3/lib/python3.7/site-packages/pykcorr-0.2.1-py3.7.egg/pyprosp/prospector.py:452: UserWarning: Cannot remove 'not_existing_parameter' as it doesn't exist in the current model.


That's all for the `model`.

### Build `sps`

Last step of the `prospector` initialization is to build the `sps`. As we said above, to be automatically built, `build_sps` has to be executed after `build_model`, because the `sps` depends on the `model` parameters.

This method accepts two arguments:
- `sps`: if you already have a built `sps`, you can give it here.
- `zcontinuous`: this is a `python-fsps` parameter controlling how metallicity interpolation of the SSPs is acheived (a value of 1 is recommended):
    - 0: use discrete indices (controlled by parameter "zmet")
    - 1: linearly interpolate in log Z/Z_\sun to the target metallicity (the parameter "logzsol")
    - 2: convolve with a metallicity distribution function at each age (the MDF is controlled by the parameter "pmetals")

*Note: this can take a bit of time the first time you build it, but will be much faster as long as you don't restart the kernel.*

In [12]:
prosp.build_sps(zcontinuous=1)

The `prospector` initialization is now done. Let's run the SED fitting.

### SED fitting

In [13]:
prosp.describe_run_parameters("dynesty")


#====================================#
#   Running parameters description   #
#====================================#


+-------------+
|   dynesty   |
+-------------+

    nested_bound: (optional, default: 'multi')

    nested_sample: (optional, default: 'unif')

    nested_nlive_init: (optional, default: 100)

    nested_nlive_batch: (optional, default: 100)

    nested_dlogz_init: (optional, default: 0.02)

    nested_maxcall: (optional, default: None)

    nested_walks: (optional, default: 25)

    
    There are more informations on these parameters at:
        https://dynesty.readthedocs.io/en/latest/api.html#dynesty.dynesty.DynamicNestedSampler

